In [ ]:
import json
from twarc_csv import DataFrameConverter
from twarc.expansions import ensure_flattened
import pandas as pd
from twarc_csv import dataframe_converter
from twarc_csv import CSVConverter, DataFrameConverter
import math
import emoji
from operator import itemgetter
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta
from advertools import extract_emoji
import numpy as np
import pickle

**Manual Input**: Set the path.

In [ ]:
path='C:/Users/acoub/OneDrive/Desktop/DSDM/Thesis/ChileGov/methodology_blank'
sep='/'

In [ ]:
path_scripts = path+sep+'initial'
path_data= path+sep+'data'

In [ ]:
converter = DataFrameConverter()
data = []
with open(path+sep+"data"+sep+"first_search.json") as f:
    for line in f:
        data.append(json.loads(line))
df = converter.process(data)

In [ ]:
df.shape

In [ ]:
def hash_retrieve(df):
    """
    df : dataframe of tweets
    Description: 
        The function takes as an object a df of tweets obtained via twarc and 
        returns a generator object.
    
    """

    for line, id in zip(df['entities.hashtags'], df['id']):
        if pd.isna(line):
            continue
        line = line.strip()
        data = json.loads(line)
        for hashtag in ensure_flattened(data):
            yield [hashtag['tag'], id]

In [ ]:
hashtags_df = pd.DataFrame(list(hash_retrieve(df)),
                 columns=['tweet_hashtags', 'id'])

hashtags_df = hashtags_df.groupby('id')['tweet_hashtags'].apply(lambda x: ','.join(x))
df = df.merge(hashtags_df, how='left', left_on='id', right_on='id')
df['tweet_hashtags']=df['tweet_hashtags'].str.lower()

In [ ]:
hasht=[]
for i in df.index:
    try:
        hasht.append(df['tweet_hashtags'][i].split(","))
    except:
        hasht.append([])
    
    
df['list_hashtags']=hasht

In [ ]:
hashtags=[]
for i in df.index:
    hashtags+=df['list_hashtags'][i]

**Look output**: 100 most used hashtag for your topic in tweets geolocated in Chile.

In [ ]:
from collections import Counter
count = Counter(hashtags)
  
# most_common() produces k frequently encountered
# input values and their respective counts.
most_occur = count.most_common(100)
  
print(most_occur)

In [ ]:
import numpy as np
import pandas as pd
import emoji
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer   
from nltk.stem.wordnet import WordNetLemmatizer
import nltk
import re
import matplotlib.pyplot as plt
import seaborn as sns
import string

stopwords_eng =stopwords.words("english")
stopwords_es =stopwords.words("spanish")
stopwords_all=stopwords_eng+stopwords_es

def cleanTweets(s,remove_hash):
    #Function to clean tweets, for now i am keeping emojis and hashtags. Alternative version
    if type(s)==np.float:
        return ""
    #Demojize text
#    s=emoji.demojize(s,delimiters=("", " "))
    
    #Remove new lines, etc.
    s = s.replace(r'<lb>', "\n")
    s = s.replace(r'<tab>', "\i")
    s = re.sub(r'<br */*>', "\n", s)
    s = s.replace("&lt;", "<").replace("&gt;", ">").replace("&amp;", "&")
    s = s.replace("&amp;", "&")
    s = s.replace("\n", " ")
    s = s.replace("\\n", " ")
    
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    s= emoji_pattern.sub(r'', s)
    
    
    # markdown urls
    s = re.sub(r'\(https*://[^\)]*\)', "", s)
    # normal urls
    s = re.sub(r'https*://[^\s]*', "", s)
    #s = re.sub(r'_+', ' ', s)
    s = re.sub(r'"+', '"', s)
    #Remove punctuation    
    s = re.sub('[()!?]', ' ', s)
    s = re.sub('\[.*?\]',' ', s)
    s = re.sub('\[,*?\]',' ', s)
    # custom removals
    #s = re.sub(r'@[A-Za-z0-9_]+', "", s) # replace mentions
    s = re.sub(r':[^:]+','[emoji]',s) # remove demojized text
    s= re.sub(r'[0-9]','',s)# remove digits
    
    s = s.translate(str.maketrans('', '', '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~'))
    
    s=s.lower()
    if remove_hash==True:
        s = re.sub(r'#[A-Za-z0-9_]+', "", s)
    
    #Remove stopwords
    s = ' '.join([word for word in s.split() if word not in stopwords_all])
    
    return str(s)

In [ ]:
import unidecode
df['cleaned_text'] = [cleanTweets(text,False) for text in df['text']]
df['cleaned_text']= df['cleaned_text'].apply(unidecode.unidecode)

**Look output**: 100 most used word for your topic in tweets geolocated in Chile.

In [ ]:
df['cleaned_text']=df['cleaned_text'].fillna("")
string=" ".join(list(df['cleaned_text']))
from collections import Counter
split_it = string.split()
Counter = Counter(split_it)
most_occur = Counter.most_common(200)
  
print(most_occur)

**Manual input**: Looking the most common words in chilean context, decide if you want to add new words to the list and put all the words in the list (the words that you used in the first search and also the words that you want to add).
  
Also add the start and end date for your final data set.

In [ ]:
list_of_words=['word1','word2','word3','word4','word5','word6']

In [ ]:
filename = path_data+sep+'keywords.sav'
pickle.dump((list_of_words), open(filename, 'wb'))

In [ ]:
filename = path_data+sep+'start_search_date.sav'
start_date= pickle.load(open(filename, 'rb'))

filename = path_data+sep+'end_search_date.sav'
end_date= pickle.load(open(filename, 'rb'))

**Manual input:** Add words related with your country of interest separated by an OR.

In [ ]:
str_country="(country_related_word_1 OR country_related_word_2 OR country_related_word_3)"

In [ ]:
filename = path_data+sep+'country_string.sav'
pickle.dump((str_country), open(filename, 'wb'))

Read the output and follow the instruction in case that you have to remove some words.

In [ ]:
query=""
for word in list_of_words:
    query+= word + " OR "
query=query[:len(query)-4]

query_place="("+query+")"+ " -is:retweet place_country:CL"
query_wrelated="("+query+")"+ str_country + " -is:retweet"

if len(query_place)<=1024 and len(query_wrelated)<=1024:
    print("Ready to run the next cell and search in twarc.")
else:
    print("Query too long, reduce the list of words.")

In [ ]:
with open(path_data+sep+"second_search.txt", 'w') as f:
    f.write(query_place)
    f.write("\n")
    f.write(query_wrelated)

**Manual revision:** Run the following two cells to know the total number of tweets to download and see if does not exceed the limit of your twarc acount. If is OK run also the next cell.

In [ ]:
file=path_data+sep+"second_search.txt"
%cd {path_data}
! twarc2 searches --archive --counts-only --granularity day --start-time {start_date} --end-time {end_date} {file} second_search_count.csv

In [ ]:
count=pd.read_csv(path_data+sep+'second_search_count.csv',encoding = "ISO-8859-1")
print("Tweets to download: "+str(count['day_count'].sum()))

**Download the tweets:** (Only if you have enought limit in your twarc account)

In [ ]:
%cd {path_data}
! twarc2 searches --archive --start-time {start_date} --end-time {end_date} {file} second_search.json